Method	Ease of Use	Conversion Quality	Dependencies	Best For...
1. Calibre	Easy	Excellent	External Program	The highest quality and most reliable conversions.
2. WeasyPrint	Medium	Good	Python Libraries Only	Self-contained scripts where you can't install external programs.

Method 1: The Most Robust and Recommended Approach - Using Calibre
Calibre is a powerful, free, and open-source e-book management tool. It has a command-line utility called ebook-convert that is expertly designed for high-quality conversions between dozens of formats. We can call this utility directly from Python.

Why it's the best:
It handles complex CSS, embedded fonts, tables of contents, and metadata exceptionally well.
The output quality is generally superior to pure Python library solutions.
It's extremely reliable.

Step 1: Install Calibre
You must install Calibre on your system first. Crucially, you need to ensure its command-line tools are available in your system's PATH.

Windows: During installation, make sure you check the box that says "Add calibre to system path".

macOS: The easiest way is via Homebrew: brew install --cask calibre. The command-line tools are included.

Linux: Use your package manager, e.g., sudo apt-get install calibre or sudo dnf install calibre.

To verify, open a new terminal/command prompt and type ebook-convert --version. If it shows a version number, you are ready.

In [2]:
import subprocess
import os

def convert_epub_to_pdf_calibre(epub_path, pdf_path):
    """
    Converts an EPUB file to PDF using Calibre's ebook-convert command-line tool.

    Args:
        epub_path (str): The full path to the input .epub file.
        pdf_path (str): The full path for the output .pdf file.
    
    Returns:
        bool: True if conversion was successful, False otherwise.
    """
    # Check if the input file exists
    if not os.path.exists(epub_path):
        print(f"Error: Input file not found at '{epub_path}'")
        return False

    print(f"Starting conversion of '{epub_path}' to '{pdf_path}'...")

    try:
        # The command to run. You can add more Calibre options here if needed.
        # For example, to set margins: '--pdf-page-margin-left', '72'
        command = ['ebook-convert', epub_path, pdf_path]
        
        # Run the command.
        # check=True will raise a CalledProcessError if the command returns a non-zero exit code.
        # capture_output=True and text=True will capture stdout/stderr as strings.
        result = subprocess.run(
            command, 
            check=True, 
            capture_output=True, 
            text=True
        )
        
        print("Conversion successful!")
        # print("Calibre Output:\n", result.stdout) # Uncomment for detailed output
        return True

    except FileNotFoundError:
        print("Error: 'ebook-convert' command not found.")
        print("Please ensure Calibre is installed and its command-line tools are in your system's PATH.")
        return False
    except subprocess.CalledProcessError as e:
        print("Error during conversion.")
        print(f"Calibre returned a non-zero exit code: {e.returncode}")
        print("Error Output:\n", e.stderr)
        return False

# # --- USAGE EXAMPLE ---
# if __name__ == "__main__":
#     # Make sure to replace these with your actual file paths
#     input_file = 'my-book.epub'
#     output_file = 'my-book-from-calibre.pdf'
    
#     # Create a dummy epub file for testing if you don't have one
#     if not os.path.exists(input_file):
#         print(f"Creating a dummy file '{input_file}' for demonstration.")
#         with open(input_file, 'w') as f:
#              # This is not a valid epub, but it's enough to test the error handling
#              f.write("This is a placeholder.")
#         print("Please replace the dummy file with a real .epub file.")
#     else:
#         convert_epub_to_pdf_calibre(input_file, output_file)

In [3]:
input_path = r"C:\Users\laxmidhar.routa\Downloads\Build a Large Language Model (From Scratch) -- Sebastian Raschka -- 2024 -- Manning Publications Co_ LLC.epub"
output_path = "Build a Large Language Model (From Scratch) -- Sebastian Raschka -- 2024 -- Manning Publications Co_ LLC.pdf"
convert_epub_to_pdf_calibre(input_path, output_path)

Starting conversion of 'C:\Users\laxmidhar.routa\Downloads\Build a Large Language Model (From Scratch) -- Sebastian Raschka -- 2024 -- Manning Publications Co_ LLC.epub' to 'Build a Large Language Model (From Scratch) -- Sebastian Raschka -- 2024 -- Manning Publications Co_ LLC.pdf'...
Error: 'ebook-convert' command not found.
Please ensure Calibre is installed and its command-line tools are in your system's PATH.


False

In [ ]:
Method 2: A Pure Python Approach - Using ebooklib and WeasyPrint
This method does not require any external program installation (like Calibre), only Python libraries. It works by:
Using ebooklib to parse the EPUB file and access its contents (HTML chapters, CSS, images).
Using BeautifulSoup to process the HTML and embed images directly into the markup.
Using WeasyPrint to render the final, combined HTML into a high-quality PDF.
Step 1: Install Required Python Libraries
Generated bash
pip install ebooklib WeasyPrint beautifulsoup4

Conclusion
For the best results and easiest implementation, use Method 1 (Calibre). The quality of its conversion engine is hard to beat.
If you need a self-contained Python script with no external program dependencies, use Method 2 (WeasyPrint). It's a very capable solution but may require tweaking the code for particularly complex EPUB files.

In [ ]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import base64
import os
from weasyprint import HTML, CSS

def convert_epub_to_pdf_weasyprint(epub_path, pdf_path):
    """
    Converts an EPUB file to PDF using pure Python libraries.

    Args:
        epub_path (str): The full path to the input .epub file.
        pdf_path (str): The full path for the output .pdf file.
    
    Returns:
        bool: True if conversion was successful, False otherwise.
    """
    if not os.path.exists(epub_path):
        print(f"Error: Input file not found at '{epub_path}'")
        return False

    print(f"Loading EPUB from '{epub_path}'...")
    book = epub.read_epub(epub_path)
    
    # Combine all content into a single HTML string
    full_html_content = ""
    
    # Store CSS and image data
    styles = ""
    resources = {}

    # First, gather all CSS and binary resources (images, fonts)
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_STYLE:
            styles += item.get_content().decode('utf-8')
        elif item.get_type() in (ebooklib.ITEM_IMAGE, ebooklib.ITEM_FONT):
            # Store the resource content encoded in base64
            mime_type = item.get_media_type()
            data = base64.b64encode(item.get_content()).decode('utf-8')
            resources[item.get_name()] = f"data:{mime_type};base64,{data}"

    # Process documents in the order defined by the book's spine
    print("Processing book chapters...")
    for item in book.spine:
        doc = book.get_item_with_href(item[0])
        content = doc.get_content().decode('utf-8')
        
        # Use BeautifulSoup to parse and manipulate the HTML
        soup = BeautifulSoup(content, 'html.parser')
        
        # Replace image paths with embedded base64 data
        for img_tag in soup.find_all('img'):
            src = img_tag.get('src')
            if src and src in resources:
                img_tag['src'] = resources[src]
        
        full_html_content += str(soup)

    print("Generating PDF...")
    try:
        # Create a WeasyPrint HTML object
        html = HTML(string=full_html_content)
        
        # Create a WeasyPrint CSS object
        css = CSS(string=styles)
        
        # Render the PDF
        html.write_pdf(pdf_path, stylesheets=[css])
        
        print(f"Successfully converted and saved to '{pdf_path}'")
        return True
    except Exception as e:
        print(f"An error occurred during PDF generation: {e}")
        return False


# --- USAGE EXAMPLE ---
if __name__ == "__main__":
    # Make sure to replace these with your actual file paths
    input_file = 'my-book.epub'
    output_file = 'my-book-from-weasyprint.pdf'

    if not os.path.exists(input_file):
        print(f"Creating a dummy file '{input_file}' for demonstration.")
        with open(input_file, 'w') as f:
             f.write("This is a placeholder.")
        print("Please replace the dummy file with a real .epub file.")
    else:
        convert_epub_to_pdf_weasyprint(input_file, output_file)